# Load Prompt Registry Client
In the next example, following the previous Notebook on Prompt creation, we will create and upload a Prompt template to the Prompt Registry

In [1]:
from gen_ai_hub.proxy import get_proxy_client
from gen_ai_hub.prompt_registry.client import PromptTemplateClient
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

from gen_ai_hub.prompt_registry.models.prompt_template import PromptTemplateSpec, PromptTemplate
from dotenv import load_dotenv
load_dotenv()

proxy_client = get_proxy_client(proxy_version="gen-ai-hub")
prompt_registry_client = PromptTemplateClient(proxy_client=proxy_client)
print("PromptTemplateClient initialized successfully.")

prompt_template_spec = PromptTemplateSpec(
    template=[
        PromptTemplate(
            role="system",
            content="You are a helpful translation assistant."),
        PromptTemplate(
            role="user",
            content="Translate the following text to {{?to_lang}}: {{?text}}"
        )],
    defaults={'to_lang': 'German'},
)

response = prompt_registry_client.create_prompt_template(
    scenario="translation",
    name="translation_prompt",
    version="1.0.0",
    prompt_template_spec=prompt_template_spec
)
print("Prompt template created successfully.")

template_id = response.id
print(f"Prompt template ID: {template_id}")

PromptTemplateClient initialized successfully.
Prompt template created successfully.
Prompt template ID: bad89858-ec35-4f6c-9189-12a0e35a4daa


# Retrieve Prompt Template 

In [2]:
from gen_ai_hub.orchestration.models.template_ref import TemplateRef


# Fetching the template using ID 
response = prompt_registry_client.get_prompt_template_by_id(template_id)


if response and response.spec:
    template = response.spec.template
    print(f" Retrieved template: {template}")
else:
    print("Failed to retrieve template.")

 Retrieved template: [PromptTemplate(role='system', content='You are a helpful translation assistant.'), PromptTemplate(role='user', content='Translate the following text to {{?to_lang}}: {{?text}}')]


# Define the LLM
The `LLM` class is used to configure and initialize a language model for generating text based on specific parameters. In this example, we'll use the gpt-4o

In [3]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 256, "temperature": 0.2})

# Create Orchestration Configuration
The `OrchestrationConfig` class defines a configuration for integrating various modules, such as templates and language models, into a cohesive orchestration setup.

In [4]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

response = prompt_registry_client.get_prompt_template_by_id(template_id)
template = response.spec.template

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

# Run the Orchestration Request

In [5]:
from gen_ai_hub.orchestration.service import OrchestrationService
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

orchestration_service = OrchestrationService(config=config)

result = orchestration_service.run(template_values=[
    TemplateValue(name="text", value="The Orchestration Service is working!")
])

print(result.orchestration_result.choices[0].message.content)

AttributeError: 'list' object has no attribute 'to_dict'